In [1]:
import pyfdb

In [8]:
import sys
import os
sys.path.append("../gsv_interface")

from gsv.retriever import GSVRetriever

gsv = GSVRetriever()  # Uncomment to request from YAML

In [3]:
request = { 'class': 'ea',
    'domain': 'g',
    'type': 'an',
    'stream': 'moda',
    'date': '19910401',
    'levelist': '500',
    'levtype': 'pl',
    'param': '129',
    'expver': '0001',
}
ds = gsv.request_data(request)

Correctly decoded z      on level 500    at 1991-04-01T00:00:00


In [15]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys

def make_date_list(start, end, step="M", numsteps=1):

    if step == "M":
        delta = relativedelta(months=numsteps)
    elif step == "D":
        delta = relativedelta(days=numsteps)
    else:
        sys.exit("step not recognized")        

    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')

    date_list = []
    current_date = start_date

    while current_date <= end_date:
        date_string = current_date.strftime('%Y%m%d')
        date_list.append(date_string)
        current_date += delta

    return date_list

In [17]:
def GSV_generator(gsv, request, start, end):
    """Return GSV request as a generator from a date list"""
    date_list= make_date_list(start, end)
    for date in date_list:
        request["date"] = date
        yield gsv.request_data(request)
        



In [18]:
request = { 
        'class': 'ea',
        'domain': 'g',
        'type': 'an',
        'stream': 'moda',
        'date': '19910401',
        'levelist': '500',
        'levtype': 'pl',
        'param': '129',
        'expver': '0001',
    }

gen = GSV_generator(gsv, request, '20000101', '20201201')

In [19]:
for ds in gen:
    print(ds.time)

Correctly decoded z      on level 500    at 2000-01-01T00:00:00
<xarray.DataArray 'time' (time: 1)>
array(['2000-01-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01
Correctly decoded z      on level 500    at 2000-02-01T00:00:00
<xarray.DataArray 'time' (time: 1)>
array(['2000-02-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-02-01
Correctly decoded z      on level 500    at 2000-03-01T00:00:00
<xarray.DataArray 'time' (time: 1)>
array(['2000-03-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-03-01
Correctly decoded z      on level 500    at 2000-04-01T00:00:00
<xarray.DataArray 'time' (time: 1)>
array(['2000-04-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-04-01
Correctly decoded z      on level 500    at 2000-05-01T00:00:00
<xarray.DataArray 'time' (time: 1)>
arra